# Machine Learning
    https://www.youtube.com/watch?v=OGxgnH8y2NM
    https://www.youtube.com/watch?v=JcI5Vnw0b2c#t=2.98320246
  

    pip3 install sklearn
    pip3 install quandl
    pip3 install pandas

## Regresion

In [1]:
import pandas as pd
import quandl
df = quandl.get('WIKI/GOOGL')
print(df.head())

              Open    High     Low   Close      Volume  Ex-Dividend  \
Date                                                                  
2004-08-19  100.00  104.06   95.96  100.34  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.31  22834300.0          0.0   
2004-08-23  110.75  113.48  109.05  109.40  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.87  15247300.0          0.0   
2004-08-25  104.96  108.00  103.88  106.00   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2004-08-19          1.0     50.000      52.03    47.980      50.170   
2004-08-20          1.0     50.505      54.54    50.250      54.155   
2004-08-23          1.0     55.375      56.74    54.525      54.700   
2004-08-24          1.0     55.620      55.80    51.785      52.435   
2004-08-25          1.0     52.480      54.00    51.940      53.000   

    

In [2]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19      50.170  3.707395    0.340000   44659000.0
2004-08-20      54.155  0.710922    7.227007   22834300.0
2004-08-23      54.700  3.729433   -1.218962   18256100.0
2004-08-24      52.435  6.417469   -5.726357   15247300.0
2004-08-25      53.000  1.886792    0.990854    9188600.0


## Características y etiquetas

In [3]:
import math
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)
forecast_out = int(math.ceil(0.01*len(df)))  # desplazamiento hacia arriba
print("forecast_out =", forecast_out)
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
print(df.head())

forecast_out = 30
            Adj. Close    HL_PCT  PCT_change  Adj. Volume   label
Date                                                             
2004-08-19      50.170  3.707395    0.340000   44659000.0  66.290
2004-08-20      54.155  0.710922    7.227007   22834300.0  67.530
2004-08-23      54.700  3.729433   -1.218962   18256100.0  69.185
2004-08-24      52.435  6.417469   -5.726357   15247300.0  68.540
2004-08-25      53.000  1.886792    0.990854    9188600.0  69.425


## Training and Testing

(nota: se recompiló `numpy` con `easy_install --upgrade numpy`)

In [4]:
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])
X = preprocessing.scale(X)
print(len(X), len(y))

2932 2932


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [5]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.960745194098


In [6]:
clf = svm.SVR(kernel='poly')
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy) # no tan bueno!

0.676831908822


In [7]:
clf = LinearRegression(n_jobs=-1)  # tantos como aguante el procesador
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy) 

0.960745194098


In [8]:
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately  = X[-forecast_out:]
X = X[:-forecast_out]

df.dropna(inplace=True)
y = np.array(df['label'])
y = y[:-forecast_out]  # ajusta tamaño de y

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)

#pickle
# Pickling and Scaling
# pickling == serialización
# objetivo: evitar entrenar el clasificador varias veces, se guarda en un archivo una vez
# entrenado
import pickle
# aquí el clasificador ya está entrenado, así lo guardaremos

with open("linearregression.pickle", "wb") as f:
    pickle.dump(clf, f)

pickle_in = open('linearregression.pickle', 'rb')
clf = pickle.load(pickle_in)

# fin de pickle  
# una vez guardado, se puede omitir el código desde la linea clf = LinearRegression()
# se puede crear el clasificador en equipos poderosos y recuperar el pickle en una
# máquina mas pequeña para usarlo.


accuracy = clf.score(X_test, y_test)
forecast_set = clf.predict(X_lately)
print(forecast_set, accuracy, forecast_out)

[ 762.23648159  759.57276181  766.8844566   764.59657194  762.01096211
  759.64697089  758.26632401  770.0137785   771.63576156  767.50878702
  774.08678168  769.44584431  763.33890722  772.8195701   764.17202393
  764.01341722  761.59842633  769.37313119  776.3189003   779.94816142
  784.80878598  792.12769485  779.2010205   779.55915176  784.35874612
  737.91802424  746.56723447  727.7138098   725.91731826  706.41330142] 0.96265172885 30


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [9]:
import matplotlib.pyplot as plt
from matplotlib import style
import datetime

style.use('ggplot')
df['Forecast'] = np.nan

#obtiene ultima fecha +1 
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

# Teoría de regresión

## regresión lineal

y = mx+b

m = ((x'y')-(xy)') / ( (x')^2 - (x^2)' )

donde x' es la media de x (promedio)

b = y' - m*(x')

(ajuste a puntos que parecen formar una recta)


In [10]:
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

xs = np.array([1,2,3,4,5,6], dtype=np.float64)
ys = np.array([5,4,6,5,6,7], dtype=np.float64)

#plt.scatter(xs, ys)
#plt.show()

def best_fit_slope_and_intercept(xs, ys):
    m = ( (mean(xs) * mean(ys) - mean(xs * ys)) /
         (mean(xs)**2 - mean(xs**2)) )
    b = mean(ys) - m * mean(xs)
    return m, b

m, b = best_fit_slope_and_intercept(xs, ys)

pred_x = 8
pred_y = m * pred_x + b

regression_line = [m*x+b for x in xs]
plt.scatter(xs, ys)
plt.scatter(pred_x, pred_y, color='g')

plt.plot(xs, regression_line)
plt.show()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


## error (al cuadrado)

r^2 = 1 - (SE y_fit)/ (SE y')

In [11]:
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

xs = np.array([1,2,3,4,5,6], dtype=np.float64)
ys = np.array([5,4,6,5,6,7], dtype=np.float64)

#plt.scatter(xs, ys)
#plt.show()

def best_fit_slope_and_intercept(xs, ys):
    m = ( (mean(xs) * mean(ys) - mean(xs * ys)) /
         (mean(xs)**2 - mean(xs**2)) )
    b = mean(ys) - m * mean(xs)
    return m, b

def squared_error(ys_orig, ys_line):  # SE
    return sum((ys_line - ys_orig)**2)

def coefficient_of_determination(ys_orig, ys_line):
    y_mean_line = [mean(ys_orig)] * len(ys_orig)  # horizontal
    squared_error_regr = squared_error(ys_orig, ys_line)
    squared_error_y_mean = squared_error(ys_orig, y_mean_line)
    return 1 - (squared_error_regr / squared_error_y_mean)

m, b = best_fit_slope_and_intercept(xs, ys)

# predictions
pred_x = 8
pred_y = m * pred_x + b

# squared error
r_squared = coefficient_of_determination(ys, regression_line)
print(r_squared)

regression_line = [m*x+b for x in xs]
plt.scatter(xs, ys)
plt.scatter(pred_x, pred_y, color='g')

plt.plot(xs, regression_line)
plt.show()

0.584415584416


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


datos de ejemplo

In [12]:
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import random

style.use('fivethirtyeight')

#xs = np.array([1,2,3,4,5,6], dtype=np.float64)
#ys = np.array([5,4,6,5,6,7], dtype=np.float64)

#plt.scatter(xs, ys)
#plt.show()

def best_fit_slope_and_intercept(xs, ys):
    m = ( (mean(xs) * mean(ys) - mean(xs * ys)) /
         (mean(xs)**2 - mean(xs**2)) )
    b = mean(ys) - m * mean(xs)
    return m, b

def squared_error(ys_orig, ys_line):  # SE
    return sum((ys_line - ys_orig)**2)

def coefficient_of_determination(ys_orig, ys_line):
    y_mean_line = [mean(ys_orig)] * len(ys_orig)  # horizontal
    squared_error_regr = squared_error(ys_orig, ys_line)
    squared_error_y_mean = squared_error(ys_orig, y_mean_line)
    return 1 - (squared_error_regr / squared_error_y_mean)

def create_dataset(hm, variance, step=2, correlation=False):
    val = 1
    ys = []
    for i in range(hm):
        y = val + random.randrange(-variance, variance)
        ys.append(y)
        if correlation and correlation == 'pos':
            val += step
        elif correlation and correlation == 'neg':
            val -= step
    xs = range(hm)
    return np.array(xs, dtype=np.float64), np.array(ys, dtype=np.float64)

xs, ys = create_dataset(40, 10, 2, False)

m, b = best_fit_slope_and_intercept(xs, ys)

# predictions
pred_x = xs[-1] + 1
pred_y = m * pred_x + b

regression_line = [m*x+b for x in xs]

# squared error
r_squared = coefficient_of_determination(ys, regression_line)
print(r_squared)

plt.scatter(xs, ys)
plt.scatter(pred_x, pred_y, s=100, color='g')

plt.plot(xs, regression_line)
plt.show()

0.00366506658335


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [21]:
import pandas as pd
import quandl, math, datetime
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl

style.use('ggplot')
mpl.rcParams['lines.linewidth'] = 0.5

df = quandl.get('WIKI/GOOGL')
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0


df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]

forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)
forecast_out = int(math.ceil(0.1*len(df)))  # desplazamiento hacia arriba
df['label'] = df[forecast_col].shift(-forecast_out)

X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately  = X[-forecast_out:]
X = X[:-forecast_out]

df.dropna(inplace=True)
y = np.array(df['label'])

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

clf = LinearRegression()
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
forecast_set = clf.predict(X_lately)

df['Forecast'] = np.nan

#obtiene ultima fecha +1 
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


http://archive.ics.uci.edu/ml/datasets.html

## clasificación por medio de los k - vecinos cercanos

Obtiene los k vecinos más cercanos al punto a clasificar desde los datos, el punto se clasificará con la clase que tenga la mayoría de estos k-vecinos

In [40]:
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

df = pd.read_csv('breast-cancer-wisconsin.data.txt')
df.replace('?', -99999, inplace=True)
df.drop(['id'], 1, inplace=True)

X = np.array(df.drop(['class'], 1))
y = np.array(df['class'])

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

clf = neighbors.KNeighborsClassifier()  # default k=5
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
print(accuracy)

example_measures = np.array([[4,2,1,1,1,2,3,2,1], [4,2,1,2,4,2,3,2,1]])
example_measures = example_measures.reshape(len(example_measures),-1)

prediction = clf.predict(example_measures)
print(prediction)

0.971428571429
[2 2]


## funcionamiento interno de k-vecinos cercanos

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from matplotlib import style

style.use('fivethirtyeight')

dataset = {'k':[[1,2],[2,3],[3,1]], 'r':[[6,5],[7,7],[8,6]]}
new_features = [5,7]

def k_nearest_neighbors(data, predict, k=3):
    if len(data) >= k:
        warnings.warn('K is set to a value less than the total voting groups!')
    distances = []
    for group in data:
        for features in data[group]:
            euclidean_distance = np.linalg.norm(np.array(features)- np.array(predict))
            distances.append([euclidean_distance, group])
    votes = [i[1] for i in sorted(distances)[:k]] #ordena y toma los primeros k votos
    vote_result = Counter(votes).most_common(1)[0][0]
    
    return vote_result

result = k_nearest_neighbors(dataset, new_features, k=3)
print(result)

[[plt.scatter(*ii, s=100, color=i) for ii in dataset[i]] for i in dataset]
plt.scatter(*new_features, color=result)
plt.show()

r


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [46]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from matplotlib import style
import pandas as pd
import random

style.use('fivethirtyeight')

dataset = {'k':[[1,2],[2,3],[3,1]], 'r':[[6,5],[7,7],[8,6]]}
new_features = [5,7]

def k_nearest_neighbors(data, predict, k=3):
    if len(data) >= k:
        warnings.warn('K is set to a value less than the total voting groups!')
    distances = []
    for group in data:
        for features in data[group]:
            euclidean_distance = np.linalg.norm(np.array(features)- np.array(predict))
            distances.append([euclidean_distance, group])
    votes = [i[1] for i in sorted(distances)[:k]] #ordena y toma los primeros k votos
    vote_result = Counter(votes).most_common(1)[0][0]
    
    return vote_result

df = pd.read_csv("breast-cancer-wisconsin.data.txt")
df.replace('?', -99999, inplace=True)
df.drop(['id'], 1, inplace=True)
full_data = df.astype(float).values.tolist()  #convierte datos a floats
random.shuffle(full_data)



test_size = 0.2  #  %
train_set = {2:[], 4:[]}
test_set = {2:[], 4:[]}
train_data = full_data[:-int(test_size*len(full_data))]  # datos sin el último 20%
test_data = full_data[-int(test_size*len(full_data)):]  # datos en el último 20%

for i in train_data:
    train_set[i[-1]].append(i[:-1])

for i in test_data:
    test_set[i[-1]].append(i[:-1])

correct = 0
total = 0

for group in test_set:
    for data in test_set[group]:
        vote = k_nearest_neighbors(train_set, data, k=5)
        if group == vote:
            correct += 1
        total += 1

print('Accuracy:', correct/total)

Accuracy: 0.9856115107913669
